In [ ]:
%pip install ydata-profiling
%pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.3 MB 335.7 kB/s eta 0:00:06
   -------- ------------------------------- 0.5/2.3 MB 335.7 kB/s eta 0:00:06
   -------- ------------------------------- 0.5/2.3 MB 335.7 kB/s eta 0:00:06
   ------------- -------------------------- 0.8/2.3 MB 372.9 kB/s eta 0:00:05
   ------------- -------------------------- 0.8/2.3 MB 372.9 kB/s eta 0:00:05
   ----------------- ---------------------- 1.0/2.3 MB 390.1 kB/s eta 0:00:04
   ----------------- -------------------

# Análisis Exploratorio

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport

# Cargar el archivo CSV
df_train = pd.read_csv('train.csv')

# Generar el reporte de análisis exploratorio y mostrarlo en el notebook
profile = ProfileReport(df_train, title="Exploratory Data Analysis Report", explorative=True)

# Mostrar el reporte dentro del Jupyter Notebook
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Limpieza de datos

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk

nltk.download('punkt')
nltk.download('stopwords')


def limpiar_texto(texto):
    if texto is None:
        return ""
    texto = texto.lower()
    texto = re.sub(r"[#@']", "", texto)
    texto = re.sub(r"http\S+|www\S+|https\S+", "", texto, flags=re.MULTILINE)
    texto = texto.encode('ascii', 'ignore').decode('ascii')
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(texto)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    texto_limpio = ' '.join(tokens)
    return texto_limpio



In [ ]:
import json
def latex_a_texto(latex):
    texto = re.sub(r'\\begin\{itemize\}', '', latex)
    texto = re.sub(r'\\end\{itemize\}', '', texto)
    texto = re.sub(r'\\item\s*', '\n', texto)
    texto = re.sub(r'\\[a-zA-Z]+\{[^}]*\}', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def convertir_a_lista(texto):
    try:
        return json.loads(texto)
    except json.JSONDecodeError:
        return texto

In [ ]:
import ast 
df_train['prompt'] = df_train['prompt'].apply(ast.literal_eval)

In [ ]:
df_train['prompt_limpio'] = df_train['prompt'].apply(lambda lista: [limpiar_texto(texto) for texto in lista])

In [ ]:
df_train['response_a_texto'] = df_train['response_a'].apply(latex_a_texto)
df_train['response_b_texto'] = df_train['response_b'].apply(latex_a_texto)

In [ ]:
df_train['response_a_texto'] = df_train['response_a_texto'].apply(convertir_a_lista)
df_train['response_b_texto'] = df_train['response_b_texto'].apply(convertir_a_lista)

Probando leer una conversación entre el usuario y el modelo:

In [ ]:
for i in range(len(df_train["prompt"][0])):
  print(i)
  print(df_train["prompt"][0][i]) 
  print("\n")
  print(df_train["response_a_texto"][0][i])
  print("\n")
  print(df_train["response_b_texto"][0][i])
  print("\n")

Terminando de limpiar:

In [ ]:
df_train['response_a_limpio']=df_train['response_a_texto'].apply(lambda lista: [limpiar_texto(texto) for texto in lista])

In [ ]:
df_train['response_b_limpio']=df_train['response_b_texto'].apply(lambda lista: [limpiar_texto(texto) for texto in lista])